# Импорт библиотек

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/MTS ML Cup

/content/drive/MyDrive/MTS ML Cup


In [4]:
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [ ]:
!pip install implicit

In [ ]:
!pip install catboost

In [ ]:
!pip install polars

In [13]:
import pandas as pd
import numpy as np
import scipy
import implicit
import polars as pl

In [ ]:
!pip install feather-format >> none

In [14]:
RAND = 42

# Описание задачи

**В данном jupyter-ноутбуке на основании признаков url_host, region_name, city_name, cpe_model_name мы будем создавать векторные пространства (эмбеддинги), чтобы сравнивать схожесть пользователей по новым полученным признакам и в дальнейшем использовать эти эмбеддинги, как признаки для наших моделей**

## О соревновании и данных

https://ods.ai/competitions/mtsmlcup

Задача соревнования
- Определение пола и возраста владельца HTTP cookie по истории активности пользователя в интернете на основе синтетических данных.

Метрики соревнования:
* ROC-AUC – для определения пола, f1 weighted – для определения возраста.
* Все решения рассчитываются по формуле -  2 * f1_weighted(по 6 возрастным бакетам) + gini по полу.
* Возрастные бакеты (Класс 1 — 19-25, Класс 2 — 26-35, Класс 3 — 36-45, Класс 4 — 46-55, Класс 5 — 56-65, Класс 6 — 66+).

**Данные с признаками для этой части скачаны с:**
* https://www.kaggle.com/datasets/nfedorov/mts-ml-cookies

**Это те же сырые данные, но в формате feather.**

Описание колонок файла с данными:
* 'region_name' – Регион
* 'city_name' – Населенный пункт
* 'cpe_manufacturer_name' – Производитель устройства
* 'cpe_model_name' – Модель устройства
* 'url_host' – Домен, с которого пришел рекламный запрос
* 'cpe_type_cd' – Тип устройства (смартфон или что-то другое)
* 'Cpe_model_os_type' – Операционка на устройстве
* 'price' – Оценка цены устройства
* 'date' – Дата
* 'part_of_day' – Время дня (утро, вечер, и тд)
* 'request_cnt' – Число запросов одного пользователя за время дня (поле part_of_day)
* 'user_id' – ID пользователя

Описание колонок файла с таргетами:

* 'age' – Возраст пользователя
* 'is_male' – Признак пользователя : мужчина (1-Да, 0-Нет)
* 'user_id' – ID пользователя

## Структура проекта

Данная работа была разделена на несколько jupyter ноутбуков:

0. Data_preparing.ipnb - аггрегация отдельных файлов по user_id и склейка в финальный датасет
1. EDA.ipynb - исследовательская часть
2. baseline.ipynb - бейзлайн модели
3. create_embeddings.ipynb - создание эмбеддингов для дальнейшего их использования в качестве фич
4. baseline_embeddings.ipynb - бейзлан модели с эмбеддингами
5. model_tuning.ipynb - тюнинг наиболее перспективных моделей
6. gender_prediction_stacking.ipynb - стекинг моделей для предсказания пола

# Загрузка и обработка данных

In [8]:
# сырые данные с признаками
data = pd.read_feather('dataset_full.feather')
data.head(3)

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,20368.0,2022-06-15,morning,1,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,20368.0,2022-06-19,morning,1,45098
2,Краснодарский край,Краснодар,Apple,iPhone 7,avatars.mds.yandex.net,smartphone,iOS,20368.0,2022-06-12,day,1,45098


In [16]:
# мы будем делать эмбеддинги по признакам 
# url_host, cpe_model_name, region_name, city_name
# поэтому оставим только их, чтобы экономить память
data = data[['user_id', 'url_host', 'cpe_model_name', 
             'region_name', 'city_name', 'request_cnt']]

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 6 columns):
 #   Column          Dtype   
---  ------          -----   
 0   user_id         int32   
 1   url_host        category
 2   cpe_model_name  category
 3   region_name     category
 4   city_name       category
 5   request_cnt     int8    
dtypes: category(4), int32(1), int8(1)
memory usage: 4.2 GB


In [18]:
# кодировка категориальных переменных
data['region_name'] = data['region_name'].cat.codes
data['city_name'] = data['city_name'].cat.codes
data['cpe_model_name'] = data['cpe_model_name'].cat.codes
data['url_host'] = data['url_host'].cat.codes

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 6 columns):
 #   Column          Dtype
---  ------          -----
 0   user_id         int32
 1   url_host        int32
 2   cpe_model_name  int16
 3   region_name     int8 
 4   city_name       int16
 5   request_cnt     int8 
dtypes: int16(2), int32(2), int8(2)
memory usage: 4.2 GB


In [ ]:
# переведем данные в формат polars dataframe
data = pl.from_pandas(data)

## Embeddings по URL
Создаем эмбеддинги по url, чтобы в дальнейшем использовать их, как фичи

In [ ]:
!pip install faiss-cpu --no-cache

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# считаем кол-во запросов пользователей по url
data_agg = data.groupby(['user_id', 'url_host'],
                        maintain_order=True).agg(pl.sum("request_cnt"))

In [ ]:
# кол-во уникальных url
url_set = set(data_agg['url_host'].to_pandas())
print(f'{len(url_set)} urls')
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}

# кол-во уникальных пользователей
usr_set = set(data_agg['user_id'].to_pandas())
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

199683 urls
415317 users


In [ ]:
# создаем матрицу взаимодействия пользователя с url
%%time
values = np.array(data_agg['request_cnt'].to_pandas())
rows = np.array(data_agg['user_id'].to_pandas().map(usr_dict))
cols = np.array(data_agg['url_host'].to_pandas().map(url_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)),
                              shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(
    factors=150,
    iterations=50,
    use_gpu=False,
    calculate_training_loss=True,
    regularization=0.1)

CPU times: user 2.71 s, sys: 967 ms, total: 3.68 s
Wall time: 3.78 s


In [ ]:
%%time
als.fit(mat)

In [ ]:
u_factors = als.model.user_factors
d_factors = als.model.item_factors

In [ ]:
# соотносим полученные признаки с user_id
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
usr_emb = pd.DataFrame(u_factors)
usr_emb['user_id'] = usr_emb.index.map(inv_usr_map)

In [ ]:
# сохраняем полученные признаки
usr_emb.to_csv('embeddings/url_emb.csv', index=False)

## Embeddings по region_name
Создаем эмбеддинги по региону, чтобы в дальнейшем использовать их, как фичи

In [ ]:
# считаем кол-во запросов пользователей по регионам
data_agg_reg = data.groupby(['user_id', 'region_name'], maintain_order=True) \
    .agg(pl.sum("request_cnt"))

In [ ]:
# уникальное кол-во регионов
region_set = set(data_agg_reg['region_name'].to_pandas())
print(f'{len(region_set)} regions')
region_dict = {
    reg: reg_id
    for reg, reg_id in zip(region_set, range(len(region_set)))
}

81 regions


In [ ]:
# создаем матрицу взаимодействия пользователя с регионом
%%time
values = np.array(data_agg_reg['request_cnt'].to_pandas())
rows = np.array(data_agg_reg['user_id'].to_pandas().map(usr_dict))
cols = np.array(data_agg_reg['region_name'].to_pandas().map(region_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)),
                              shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(
    factors=30,
    iterations=20,
    use_gpu=False,
    calculate_training_loss=True,
    regularization=0.1)

CPU times: user 486 ms, sys: 1.04 ms, total: 487 ms
Wall time: 482 ms


In [ ]:
%%time
als.fit(mat)

In [ ]:
u_factors = als.model.user_factors
d_factors = als.model.item_factors

In [ ]:
# соотносим полученные признаки с user_id
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
reg_emb = pd.DataFrame(u_factors)
reg_emb['user_id'] = reg_emb.index.map(inv_usr_map)

CPU times: user 558 ms, sys: 11.1 ms, total: 569 ms
Wall time: 566 ms


In [ ]:
# сохраняем полученные признаки
reg_emb.to_csv('embeddings/reg_emb.csv', index=False)

## Embeddings по city_name
Создаем эмбеддинги по городу, чтобы в дальнейшем использовать их, как фичи

In [ ]:
# считаем кол-во запросов пользователей по городам
data_agg_city = data.groupby(['user_id', 'city_name'], maintain_order=True) \
    .agg(pl.sum("request_cnt"))

In [ ]:
# уникальное кол-во городов
city_set = set(data_agg_city['city_name'].to_pandas())
print(f'{len(city_set)} cities')
city_dict = {
    city: city_id
    for city, city_id in zip(city_set, range(len(city_set)))
}

985 cities


In [ ]:
# создаем матрицу взаимодействия пользователя с городом
%%time
values = np.array(data_agg_city['request_cnt'].to_pandas())
rows = np.array(data_agg_city['user_id'].to_pandas().map(usr_dict))
cols = np.array(data_agg_city['city_name'].to_pandas().map(city_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)),
                              shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(
    factors=50,
    iterations=30,
    use_gpu=False,
    calculate_training_loss=True,
    regularization=0.1)

CPU times: user 478 ms, sys: 4 ms, total: 482 ms
Wall time: 484 ms


In [ ]:
%%time
als.fit(mat)

In [ ]:
u_factors = als.model.user_factors
d_factors = als.model.item_factors

In [ ]:
# соотносим полученные признаки с user_id
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
city_emb = pd.DataFrame(u_factors)
city_emb['user_id'] = city_emb.index.map(inv_usr_map)

CPU times: user 486 ms, sys: 11 ms, total: 497 ms
Wall time: 494 ms


In [ ]:
# сохраняем полученные признаки
city_emb.to_csv('embeddings/city_emb.csv', index=False)

## Embeddings по cpe_model_name
Создаем эмбеддинги по модели телефона, чтобы в дальнейшем использовать их, как фичи

In [ ]:
# считаем кол-во запросов пользователей по модели телефона
data_agg_model = data.groupby(['user_id', 'cpe_model_name'], maintain_order=True) \
    .agg(pl.sum("request_cnt"))

In [ ]:
# уникальное кол-во телефонов
model_set = set(data_agg_model['cpe_model_name'].to_pandas())
print(f'{len(model_set)} cities')
model_dict = {
    model: model_id
    for model, model_id in zip(model_set, range(len(model_set)))
}

599 cities


In [ ]:
# создаем матрицу взаимодействия пользователя с разных устройств
%%time
values = np.array(data_agg_model['request_cnt'].to_pandas())
rows = np.array(data_agg_model['user_id'].to_pandas().map(usr_dict))
cols = np.array(data_agg_model['cpe_model_name'].to_pandas().map(model_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)),
                              shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(
    factors=50,
    iterations=30,
    use_gpu=False,
    calculate_training_loss=True,
    regularization=0.1)

CPU times: user 400 ms, sys: 5.95 ms, total: 406 ms
Wall time: 400 ms


In [ ]:
%%time
als.fit(mat)

In [ ]:
u_factors = als.model.user_factors
d_factors = als.model.item_factors

In [ ]:
# соотносим полученные признаки с user_id
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
model_emb = pd.DataFrame(u_factors)
model_emb['user_id'] = model_emb.index.map(inv_usr_map)

CPU times: user 521 ms, sys: 11.1 ms, total: 532 ms
Wall time: 512 ms


In [ ]:
# сохраняем полученные признаки
model_emb.to_csv('embeddings/model_emb.csv', index=False)